In [13]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('ostra-dd1177d64769.json', scope)
client = gspread.authorize(creds)

sheet = client.open("dictionary").sheet1

### List cells

In [14]:
list_all = sheet.get_all_records()
for index, word in enumerate(list_all):
    print(index+1, word)

1 {'en': 'bike', 'ru': 'велосипед', 'es': 'bicicleta'}


### Update cell

In [15]:
class Translator():
    
    def __init__(self, sheet):
        self.sheet = sheet
        self.map_language = {index+1: language for index, language in enumerate(sheet.row_values(1))}
        self.to_languages = [{"language": language, "index": index} for index, language in self.map_language.items()]
        
    def show(self):
        list_all = self.sheet.get_all_records()
        return list_all
    
    def translate(self, row, col, word, from_language, to_language):
        content = f'=GOOGLETRANSLATE("{word}", "{from_language}", "{to_language["language"]}")'
        self.sheet.update_cell(row, col, word)
        self.sheet.update_cell(row, to_language["index"], content)
    
    def translate_row(self, row, col, word):
        from_language = self.map_language[col]
        to_languages = [elt for elt in self.to_languages if elt["language"]!=from_language]
        for to_language in to_languages:
            self.translate(row, col, word, from_language, to_language)
            
    def delete(self, row, col):
        self.sheet.update_cell(row, col, "")
        
    def delete_row_(self, row):
        self.sheet.delete_row(row)
        
    def clean(self):
        list_all = self.show()
        rows = []
        for index, list_ in enumerate(list_all):
            if not any(word.strip() for word in list(list_.values())):
                rows.append(index+1)
        for row in sorted(rows, reverse=True):
            self.delete_row_(row)

In [22]:
translator = Translator(sheet)

translator.map_language
translator.translate_row(3, 1, "the train")
translator.show()

[{'en': 'plane', 'ru': 'самолет', 'es': 'avión'},
 {'en': 'the train', 'ru': 'поезд', 'es': 'el tren'}]

In [23]:
translator.show()

[{'en': 'plane', 'ru': 'самолет', 'es': 'avión'},
 {'en': 'the train', 'ru': 'поезд', 'es': 'el tren'}]